In [74]:
import pandas as pd
import numpy as np
import random

In [75]:
dataset = pd.read_csv("./datasets/multiclass.csv")
X = dataset.iloc[:,:-1].values
Y = dataset.iloc[:,2].values

In [76]:
def format(x, y):
    dick = {
        'x1': x[0],
        'x2': x[1]
    }
    return (int(y), dick)
classes = [0, 1, 2]
feature_list = ["x1", "x2"]
dataset = [format(x, y) for (x, y) in zip(X, Y)]
dataset[:10]

[(1, {'x1': 4.148449748155552, 'x2': -5.349373873960868}),
 (0, {'x1': -6.948623942315413, 'x2': 9.261384080094093}),
 (0, {'x1': -5.837256039557852, 'x2': 10.588704071731597}),
 (2, {'x1': -5.734435901598429, 'x2': 0.614201160084101}),
 (1, {'x1': 2.829649796673452, 'x2': -4.04070236479507}),
 (0, {'x1': -4.701120134863366, 'x2': 7.904347583018205}),
 (2, {'x1': -6.723581425099838, 'x2': -1.5614959316858525}),
 (0, {'x1': -5.538625893350698, 'x2': 8.901553823281551}),
 (0, {'x1': -7.66131279774612, 'x2': 11.064073648658292}),
 (2, {'x1': -7.78033728810476, 'x2': -2.113026418071311})]

In [77]:
# Split feature data into train set, and test set
random.shuffle(dataset)
Train_set = dataset[:int(len(dataset) * 0.75)]
Test_set = dataset[int(len(dataset) * 0.75):]
dataset[:10]

[(2, {'x1': -6.680402206551326, 'x2': -1.5753489390588176}),
 (2, {'x1': -5.374359421101238, 'x2': -0.44941488700008303}),
 (0, {'x1': -5.519674843687177, 'x2': 9.354214545892336}),
 (1, {'x1': 3.5937185185317415, 'x2': -6.851470180887033}),
 (0, {'x1': -6.218983836376524, 'x2': 9.718364985661635}),
 (0, {'x1': -6.544115411851049, 'x2': 9.18968342946056}),
 (1, {'x1': 2.2751042604343006, 'x2': -7.325961129094093}),
 (0, {'x1': -4.332226141788491, 'x2': 9.098107690613682}),
 (1, {'x1': 2.1511114045635535, 'x2': -6.790125514745523}),
 (0, {'x1': -5.708800391089712, 'x2': 9.342260320550505})]

In [78]:
class MultiClassPerceptron():
    # Analytics values
    precision, recall, accuracy, fbeta_score = {}, {}, 0, {}

    def __init__(self, classes, feature_list, feature_data, iterations=100):
        self.classes = classes
        self.feature_list = feature_list
        self.feature_data = feature_data
        self.iterations = iterations

        # Initialize empty weight vectors, with extra BIAS term.
        self.weight_vectors = {c: np.array([0 for _ in range(len(feature_list) + 1)]) for c in self.classes}

    def train(self, train_set):
        for _ in range(self.iterations):
            for category, feature_dic in train_set:
                # Format feature values as a vector, with extra BIAS term.
                feature_list = [feature_dic[k] for k in self.feature_list]
                feature_list.append(1)
                feature_vector = np.array(feature_list) #!!!

                # Initialize arg_max value, predicted class.
                arg_max, predicted_class = 0, self.classes[0]

                # Multi-Class Decision Rule:
                for c in self.classes:
                    current_activation = np.dot(feature_vector, self.weight_vectors[c])
                    if current_activation >= arg_max:
                        arg_max, predicted_class = current_activation, c

                # Update Rule:
                if not (category == predicted_class):
                    self.weight_vectors[category] = np.add(self.weight_vectors[category], feature_vector)
                    self.weight_vectors[predicted_class] = np.subtract(self.weight_vectors[predicted_class], feature_vector)

    def predict(self, feature_dict):
        feature_list = [feature_dict[k] for k in self.feature_list]
        feature_list.append(1)
        feature_vector = np.array(feature_list)

        # Initialize arg_max value, predicted class.
        arg_max, predicted_class = 0, self.classes[0]

        # Multi-Class Decision Rule:
        for c in self.classes:
            current_activation = np.dot(feature_vector, self.weight_vectors[c])
            if current_activation >= arg_max:
                arg_max, predicted_class = current_activation, c

        return predicted_class

    def run_analytics(self, test_set):
        print("CLASSIFIER ANALYSIS: ")
        print("")
        self.calculate_precision(test_set)
        print ("")
        self.calculate_recall(test_set)
        print ("")
        self.calculate_fbeta_score()
        print ("")
        self.calculate_accuracy(test_set)

    def calculate_precision(self, test_set):
        test_classes = [f[0] for f in test_set]
        correct_counts = {c: 0 for c in test_classes}
        total_counts = {c: 0 for c in test_classes}

        for feature_dict in test_set:
            actual_class = feature_dict[0]
            predicted_class = self.predict(feature_dict[1])

            if actual_class == predicted_class:
                correct_counts[actual_class] += 1
                total_counts[actual_class] += 1
            else:
                total_counts[predicted_class] += 1


        print("PRECISION STATISTICS:")

        for c in correct_counts:
            self.precision[c] = (correct_counts[c] * 1.0) / (total_counts[c] * 1.0)
            print("%s Class Precision:" % (c), self.precision[c])

    def calculate_recall(self, test_set):
        test_classes = [f[0] for f in test_set]
        correct_counts = {c: 0 for c in test_classes}
        total_counts = {c: 0 for c in test_classes}

        for feature_dict in test_set:
            actual_class = feature_dict[0]
            predicted_class = self.predict(feature_dict[1])

            if actual_class == predicted_class:
                correct_counts[actual_class] += 1
                total_counts[actual_class] += 1
            else:
                total_counts[actual_class] += 1

        print("RECALL STATISTICS:")

        for c in correct_counts:
            self.recall[c] = (correct_counts[c] * 1.0) / (total_counts[c] * 1.0)
            print("%s Class Recall:" % (c), self.recall[c])

    def calculate_accuracy(self, test_set):
        correct, incorrect = 0, 0
        for feature_dict in test_set:
            actual_class = feature_dict[0]
            predicted_class = self.predict(feature_dict[1])

            if actual_class == predicted_class:
                correct += 1
            else:
                incorrect += 1

        print("ACCURACY:")
        print("Model Accuracy:", (correct * 1.0) / ((correct + incorrect) * 1.0))

    def calculate_fbeta_score(self):
        print("F-BETA SCORES: ")
        for c in self.precision:
            self.fbeta_score[c] = 2 * ((self.precision[c] * self.recall[c]) / (self.precision[c] + self.recall[c]))
            print("%s Class F-Beta Score:", self.fbeta_score[c])

In [79]:
classifier = MultiClassPerceptron(classes, feature_list, dataset)
classifier.train(Train_set)

In [80]:
classifier.run_analytics(Test_set)

CLASSIFIER ANALYSIS: 

PRECISION STATISTICS:
1 Class Precision: 1.0
0 Class Precision: 1.0
2 Class Precision: 1.0

RECALL STATISTICS:
1 Class Recall: 1.0
0 Class Recall: 1.0
2 Class Recall: 1.0

F-BETA SCORES: 
%s Class F-Beta Score: 1.0
%s Class F-Beta Score: 1.0
%s Class F-Beta Score: 1.0

ACCURACY:
Model Accuracy: 1.0
